In [1]:
import os

from arcgis.features import GeoAccessor
import arcpy
import sys
sys.path.append('../../src')
from geoai_retail import enrich_local as enrich
from ba_data_paths import ba_data
from geoai_retail import utils

%load_ext autoreload
%autoreload 2

In [2]:
raw_gdb = '../../data/raw/raw.gdb'
origin_fc = os.path.join(raw_gdb, 'block_groups')
origin_id_fld = 'ID'
dest_fc = os.path.join(raw_gdb, 'coffee')
dest_id_fld = 'LOCNUM'
weighting_fc = os.path.join(raw_gdb, 'block_points')

enrich_template_fc = '../../src/tests/test_data.gdb/block_groups_enrich_template'
test_origin_fc = '../../src/tests/test_data.gdb/block_groups'
test_dest_fc = '../../src/tests/test_data.gdb/coffee_pdx'

interim_gdb = '../../data/interim/interim.gdb'

In [3]:
enrich_vars_df = ba_data.enrich_vars_dataframe
enrich_vars_df.head()

,name,alias,collection_name,enrich_str,enrich_field_name
0,AGE0_CY,2018 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY
1,AGE1_CY,2018 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY
2,AGE2_CY,2018 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY
3,AGE3_CY,2018 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY
4,AGE4_CY,2018 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY


In [4]:
ba_data.set_to_usa_local()

True

In [5]:
enrich_variable_chunk_size = 200
enrich_idx = 1
tmp_features = test_origin_fc
out_gdb = arcpy.env.scratchGDB

# iterate to get all the variables 
for var_idx, chunk_vars in enumerate(utils.blow_chunks(ba_data.enrich_vars, enrich_variable_chunk_size)):

        # write the run result to the output geodatabase
        this_enrich_fc = arcpy.ba.EnrichLayer(
            in_features=tmp_features,
            out_feature_class=os.path.join(out_gdb, 'temp_enrich_fc'),
            variables=chunk_vars
        )[0]
        
        # if the first run, simply push to a new Pandas Dataframe
        if var_idx == 0:
            enrich_df = GeoAccessor.from_featureclass(this_enrich_fc)
        
        # if subsequent run, just add the new enrichement variables onto the original dataframe
        else:
            this_enrich_df = GeoAccessor.from_featureclass(this_enrich_fc)
            new_cols = this_enrich_df.columns.difference(enrich_df.columns)
            enrich_df.merge(this_enrich_df[new_cols], left_index=True, right_index=True)
            
# save out the dataframe to a feature class
enrich_fc = enrich_df.spatial.to_featureclass(os.path.join(out_gdb, f'tmp_enrich_{enrich_idx:04d}'))

enrich_fc

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

OSError: "C:\Users\joel5174\AppData\Local\Temp\2\scratch.gdb\temp_enrich_fc" does not exist